In [1]:
import ccxt

# Set up Binance API
binance = ccxt.binance()
symbol = "BTC/USDT"


In [2]:
import import_ipynb
from predictor import *

import numpy as np


importing Jupyter notebook from predictor.ipynb


In [3]:
rnn_close = RNNPredictor("../models/rnn_close.h5")
lstm_close = LSTMPredictor("../models/lstm_close.h5")
xgb_close = XGBoostPredictor("../models/xgb_close.json")
rnn_poc = RNNPredictor("../models/rnn_poc.h5")
lstm_poc = LSTMPredictor("../models/rnn_poc.h5")
xgb_poc = XGBoostPredictor("../models/xgb_poc.json")


def ROC(cp, tf):
    roc = []
    x = tf
    while x < len(cp):
        rocs = (cp[x] - cp[x - tf]) / cp[x - tf]
        roc.append(rocs)
        x += 1

    return roc


def predict(inputs, model_name, n_to_predict, features):
    if model_name == "RNN":
        model = rnn_close
    elif model_name == "LSTM":
        model = lstm_close
    elif model_name == "XGBoost":
        model = xgb_close

    # if features == 'Price of Change' or 'Price of Change' in features:
    #     roc = ROC(inputs['close'], 30)
    #     inputs['close'] = np.concatenate((np.zeros(30), roc))

    return model.predict(inputs, n_to_predict)


In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State

import plotly.graph_objs as go

import dash_bootstrap_components as dbc


C:\Users\Admin\AppData\Local\Temp\ipykernel_10092\3635243919.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Admin\AppData\Local\Temp\ipykernel_10092\3635243919.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [5]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc_css])

app.layout = html.Div(
    [
        html.H4(
            "BTC-USDT Analysis Dashboard",
            className="bg-primary text-white p-2 mb-2 text-center",
        ),
        dbc.Card(
            [
                html.Div(
                    [
                        dbc.Row(
                            [
                                dbc.Col(dbc.Label("Model"), width=2),
                                dbc.Col(
                                    html.Div(
                                        [
                                            dbc.RadioItems(
                                                id="model-name",
                                                className="btn-group",
                                                inputClassName="btn-check",
                                                labelClassName="btn btn-outline-primary",
                                                labelCheckedClassName="active",
                                                options=["LSTM", "RNN", "XGBoost"],
                                                value="LSTM",
                                            ),
                                        ],
                                        className="radio-group",
                                    ),
                                    width="auto",
                                ),
                            ]
                        ),
                        dbc.Row(
                            [
                                dbc.Col(dbc.Label("Features extraction"), width=2),
                                dbc.Col(
                                    dcc.Dropdown(
                                        id="features",
                                        options=[
                                            "Close",
                                            "Price of Change",
                                            # "RSI",
                                            # "Bolling Bands",
                                            # "Moving Average",
                                        ],
                                        value="Close",
                                        multi=True,
                                    ),
                                    width=7,
                                ),
                            ]
                        ),
                    ]
                ),
            ],
            body=True,
            style={
                "margin": "2rem 1rem 0rem",
            },
        ),
        dbc.Card(
            dbc.Tabs(
                [
                    dbc.Tab(
                        [dcc.Graph(id="live-graph")],
                        label="Chart",
                    )
                ]
            ),
            style={
                "margin": "2rem 1rem 0rem",
            },
        ),
        html.Div(id="remaining-time"),
        dcc.Interval(
            id="interval-component",
            interval=15 * 1000,  # in milliseconds
            n_intervals=0,
        ),
    ]
)


In [6]:
import pandas as pd
from plotly.subplots import make_subplots


@app.callback(
    [Output("live-graph", "figure"), Output("remaining-time", "children")],
    [
        Input("interval-component", "n_intervals"),
        Input("model-name", "value"),
        Input("features", "value"),
    ],
)
def update_graph(n, model_name, features):
    ohlcv = binance.fetch_ohlcv(symbol, "1m", limit=1000)
    df = pd.DataFrame(
        ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

    roc = ROC(df["close"], 30)
    roc = np.concatenate((np.zeros(30), roc))
    df["roc"] = roc

    input_data = (
        df[["timestamp", "roc"]]
        if "Price of Change" in features
        else df[["timestamp", "close"]]
    )
    n_to_predict = 100

    predictions = predict(input_data, model_name, n_to_predict, features)
    predictions = list(np.concatenate(predictions))

    trace = go.Candlestick(
        x=df["timestamp"],
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
        name=symbol,
    )

    trace_roc = go.Scatter(
        x=df["timestamp"],
        y=df["roc"],
        mode="lines",
        name="ROC (30)",
        line=dict(color="cornflowerblue", width=1),
    )

    latest_timestamp = df["timestamp"].max()
    new_timestamp = latest_timestamp + pd.Timedelta(minutes=1)
    df = df.append({"timestamp": new_timestamp}, ignore_index=True)

    trace_predictions = go.Scatter(
        x=df["timestamp"].iloc[-n_to_predict:],
        y=predictions,
        mode="lines",
        name="Predicted Close",
        line=dict(color="orange"),
    )

    rangeselector = {
        "buttons": list(
            [
                dict(count=1, label="1m", step="minute", stepmode="backward"),
                dict(count=2, label="2m", step="minute", stepmode="backward"),
                dict(count=5, label="5m", step="minute", stepmode="backward"),
                dict(count=1, label="1h", step="hour", stepmode="backward"),
                dict(count=2, label="2h", step="hour", stepmode="backward"),
                dict(count=5, label="5h", step="hour", stepmode="backward"),
                dict(step="all"),
            ]
        )
    }

    layout = go.Layout(
        title=f"Live OHLCV Chart - {symbol}",
        xaxis=dict(
            title="Time",
            rangeselector=rangeselector,
        ),
        yaxis=dict(title="Price", domain=[0.2, 1]),
        yaxis2=dict(title="ROC (30)", domain=[0.0, 0.2]),
        showlegend=True,
        height=700,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False,
    )

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
    fig.add_trace(trace, row=1, col=1)
    fig.add_trace(trace_predictions, row=1, col=1)
    fig.add_trace(trace_roc, row=2, col=1)
    fig.update_layout(layout)

    return fig, f"input: {features}"


# Start the app.

In [7]:
import webbrowser

if __name__ == "__main__":
    # webbrowser.open('http://127.0.0.1:8050/', new=0, autoraise=True)
    app.run_server(debug=False)
